In [12]:
import os
os.chdir(r'C:\Users\jf79\Cloned Repos\REAP-Scripts')

import myfuncs.myfuncs as mf

cwd = r'C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry'
os.chdir(cwd)
mf.read_directory()

import pandas as pd
pd.set_option('display.max_columns', None)
import re
import warnings
warnings.filterwarnings('ignore')

Your Current Directory is: C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry
Files in: ['.csv', 'aggregated_interactions.csv', 'aggregated_interactions.xlsx', 'Footfall Data - Strategic Assessment', 'footfall_data.csv', 'hf_footfall_data.csv', 'london_footfall_data.csv', 'Netcall Dashboard Data.csv', 'Netcall Interactions Master Data.csv', 'Netcall Interactions Master Data.zip', 'residents.csv', 'typical_borough_averages.csv', 'typical_hf (Q3).csv', 'typical_ward_averages.csv', 'visitors.csv', 'weekday_lon (annual).csv', 'weekend_lon (annual).csv', 'workers.csv', 'x.csv']



In [13]:
customer_columns = {
    'InteractionsKey':'interactionskey',
    'ETLTaskID':'etltaskid',
    'RowNumber':'rownumber',
    'GroupName':'groupname',
    'Type':'type',
    'Direction':'direction',
    'TimeQueued':'timequeued',
    'TimeEnded':'timeended',
    'Queue':'queue',
    'Skill':'skill',
    'Agent':'agent',
    'Username':'username',
    'UserID':'userid',
    'CustomerName':'customername',
    'CustomerContact':'customercontact',
    'QueueWait':'queuetime',
    'Duration':'duration',
    'TalkTime':'talktime',
    'CallHoldDuration':'holdtime',
    'WrapUpDuration':'wrapupduration',
    'AgentHandlingTime':'agenthandlingtime',
    'Result':'result',
    'ActivityCode':'activitycode',
    'AssistedBy':'assistedby',
    'TransferredTo':'transferredto',
    'MediaInteractionId':'interactionid',
    'Subject':'subject',
    'CaseReference':'casereference',
    'Tags':'tags',
    'Forwarded':'forwarded',
    'IPAddress':'ipaddress',
    'FormattedSource':'formattedsource',
    'ConversationID':'conversationid',
    'ConversationStartTime':'conversationstarttime',
    'ConversationEndTime':'conversationendtime',
    'ConversationDuration':'conversationduration'
}
master_columns = {
    'Interaction Id':'interactionid',
    'Type':'type',
    'Direction':'direction',
    'Time Queued':'timequeued',
    'Time Ended':'timeended',
    'Queue':'queue',
    'Skill':'skill',
    'Agent':'agent',
    'Customer Name':'customername',
    'Customer Contact':'customercontact',
    'Queue Time':'queuetime',
    'Duration':'duration',
    'Talk Time':'talktime',
    'Hold Time':'holdtime',
    'Wrap-up Duration':'wrapupduration',
    'Agent Handling Time':'agenthandlingtime',
    'Result':'result',
    'Activity Code':'activitycode',
    'Assisted By':'assistedby',
    'Transferred To':'transferredto',
    'Subject':'subject',
    'Conversation Start Time':'conversationstarttime',
    'Conversation End Time':'conversationendtime',
    'Conversation Duration':'conversationduration',
    'Agent Username':'agentusername',
    'Agent ID':'agentid',
    'Case Reference':'casereference',
    'Forwarded':'forwarded',
    'Source':'source',
    'Conversation ID':'conversationid',
    'IP Address':'ipaddress'
}

In [14]:
customer_service_data = mf.query_data(schema='Netcall', data='Interactions')
customer_service_data

Successfully imported Interactions


,InteractionsKey,ETLTaskID,RowNumber,GroupName,Type,Direction,TimeQueued,TimeEnded,Queue,Skill,Agent,Username,UserID,CustomerName,CustomerContact,QueueWait,Duration,TalkTime,CallHoldDuration,WrapUpDuration,AgentHandlingTime,Result,ActivityCode,AssistedBy,TransferredTo,MediaInteractionId,Subject,CaseReference,Tags,Forwarded,IPAddress,FormattedSource,ConversationID,ConversationStartTime,ConversationEndTime,ConversationDuration
0,1,1918260,1,Customer_Services,Email,Inbound,31/01/2022 15:30:58,23/03/2022 16:04:58,Accessible Transport,Accessible Transport Email,,,,,dennisred081@gmail.com,1224:34:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,Discarded,,,,2783869,,,,0,,Email,2655959,31/01/2022 15:30:58,23/03/2022 16:04:58,1224:34:00
1,2,1918260,2,Customer_Services,Email,Inbound,31/01/2022 15:40:00,23/03/2022 16:04:58,Accessible Transport,Accessible Transport Email,,,,,dennisred081@gmail.com,1224:24:58,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,Discarded,,,,2784043,,,,0,,Email,2656133,31/01/2022 15:40:00,23/03/2022 16:04:58,1224:24:58
2,3,1918260,3,Customer_Services,Email,Inbound,31/01/2022 15:48:58,23/03/2022 16:04:58,Accessible Transport,Accessible Transport Email,,,,,r.sl02@yahoo.co.uk,1224:16:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,Discarded,,,,2784357,Re: BLUE BADGE APPLICATION OUTCOME,,,0,,Email,2656445,31/01/2022 15:48:58,23/03/2022 16:04:58,1224:16:00
3,4,1918260,4,Customer_Services,Email,Inbound,31/01/2022 17:28:37,23/03/2022 16:04:58,Accessible Transport,Accessible Transport Email,,,,,aeomidi1@yahoo.co.uk,1222:36:21,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,Discarded,,,,2786103,Monireh Seif Omrani Blue Badge Document Requested,,,0,,Email,2658163,31/01/2022 17:28:37,23/03/2022 16:04:58,1222:36:21
4,5,1918260,5,Customer_Services,Email,Inbound,31/01/2022 17:33:30,23/03/2022 16:04:58,Accessible Transport,Accessible Transport Email,,,,,aeomidi1@yahoo.co.uk,1222:31:27,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,Discarded,,,,2786115,Monireh Seif Omrani Photo requested,,,0,,Email,2658175,31/01/2022 17:33:30,23/03/2022 16:04:58,1222:31:28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983213,1008479,3342682,6,Customer_Services,Email,Inbound,02/03/2025 14:34:01,03/03/2025 10:45:47,Cost of Living Crisis Email,Cost of Living Crisis Email,Joseph Addison,8492,8492,,1354uk@gmail.com,20:11:46,02:51:22,00:01:32,02:49:50,00:01:00,00:02:32,Replied,Cost of Living Payment Query,,,7757025,Re: Re: Council tax support,,,0,,"To/CC contains ""costoflivingteam@lbhf.gov.uk""",7387189,02/03/2025 14:34:01,03/03/2025 10:45:47,20:11:46
983214,1008480,3342682,2961,Customer_Services,Email,Inbound,25/02/2025 18:05:43,26/02/2025 11:23:22,Cost of Living Crisis Email,Cost of Living Crisis Email,Cush-Ashanti Porter,7844,7844,,elissarharb@hotmail.com,17:17:39,03:23:17,00:00:22,03:22:55,00:02:30,00:02:52,Completed,Follow Up,,,7739925,Re:,,,0,,"To/CC contains ""costoflivingteam@lbhf.gov.uk""",7370813,25/02/2025 18:05:43,26/02/2025 11:23:22,17:17:39
983215,1008481,3342682,2962,Customer_Services,Email,Inbound,25/02/2025 12:45:06,26/02/2025 11:23:05,Cost of Living Crisis Email,Cost of Living Crisis Email,Cush-Ashanti Porter,7844,7844,,redhand2020@gmail.com,22:37:59,22:30:31,00:01:38,22:28:53,00:01:01,00:02:39,Replied,Cost of Living Payment Query,,,7736963,www.selectyourcompliment.co.ukgrocery,,,0,,"To/CC contains ""costoflivingteam@lbhf.gov.uk""",7367911,25/02/2025 12:45:06,26/02/2025 11:23:05,22:37:59
983216,1008482,3342682,2,Customer_Services,Email,Inbound,28/02/2025 18:15:46,03/03/2025 14:12:07,Cost of Living Crisis Email,Cost of Living Crisis Email,Joseph Addison,8492,8492,,NoReply@lbhf.gov.uk,67:56:21,06:17:52,00:02:29,06:15:23,00:01:01,00:03:30,Replied,Cost of Living Payment Query,,,7756663,COL691767481 LBHF Crisis support advice,,,0,,"To/CC contains ""costoflivingteam@lbhf.gov.uk""",7386863,28/02/2025 18:15:46,03/03/2025 14:12:07,67:56:21


In [ ]:
root_dir = r"C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Interactions Data"
os.chdir(root_dir)

mf.read_directory()

# Create an empty DataFrame to store the combined data
master_data = pd.DataFrame()
 
# Iterate over each folder in the root directory
for folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder)
    # Ensure it's a directory
    if os.path.isdir(folder_path):
        # Iterate over each CSV file in the folder
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):  # Ensure it's a CSV file
                file_path = os.path.join(folder_path, file)
 
                # Read CSV
                df = pd.read_csv(file_path)
 
                # Add a new column with the CSV filename
                df["Source_File"] = file
 
                # Append to master DataFrame
                master_data = pd.concat([master_data, df], ignore_index=True)

Your Current Directory is: C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Interactions Data
Files in: ['desktop.ini', 'Interactions Feb 2023 - Apr 2024 (excl Cust Serv)', 'Interactions Feb 2025 (excl Cust Serv)', 'Interactions Jan 2022 - Dec 2022 (excl Cust Serv)', 'Interactions Jan 2024 - Jun 2024 (excl Cust Serv)', 'Interactions Jun 24 - Jan 25 (excl Cust Serv)', 'Interactions_Jan 2023 (LIMITED GROUPS)', 'Netcall Dashboard Data.csv', 'Netcall Interactions Master Data.csv']



In [ ]:
customer_service = customer_service_data.copy()
customer_service.rename(
    columns=customer_columns,
    inplace=True
)
customer_service = customer_service[customer_service['groupname'] == 'Customer_Services']

columns_to_drop = [
    'interactionskey','etltaskid','rownumber',
    'username','userid','tags','formattedsource'
]
customer_service.drop(columns=columns_to_drop, inplace=True)
col = customer_service.pop('interactionid')
customer_service.insert(0, 'interactionid', col)
customer_service['groupname'] = customer_service['groupname'].str.replace('Customer_Services','Customer Services')

customer_service.sort_values(
    ['timequeued'],
    ascending=True,
    inplace=True
)

drop = customer_service[customer_service['interactionid'].duplicated(keep='first')]
customer_service.drop(
    drop.index,
    inplace=True
)
customer_service.reset_index(
    drop='index',
    inplace=True
)

customer_service['timequeued'] = pd.to_datetime(customer_service['timequeued'], format='mixed', dayfirst=True)
customer_service['timeended'] = pd.to_datetime(customer_service['timeended'], format='mixed', dayfirst=True)
customer_service['Year'] = customer_service['timequeued'].dt.year
customer_service['Month'] = customer_service['timequeued'].dt.month

In [ ]:
master_df = master_data.copy()
master_df.rename(
    columns=master_columns,
    inplace=True
)

# Step 1: Split at the first dot (.)
master_df[['groupname', 'to_drop']] = master_df['Source_File'].str.split('.', expand=True)
 
# Step 2: Split at the first opening parenthesis (()
master_df[['groupname', 'to_drop']] = master_df['groupname'].str.split('(', expand=True)
 
# Step 3: Clean text (remove punctuation & normalize spaces)
master_df['groupname'] = master_df['groupname'].apply(lambda text: re.sub(r"[^\w\s]", "", text).strip())
master_df['groupname'] = master_df['groupname'].str.replace('Childrens', 'Children')
master_df['groupname'] = master_df['groupname'].str.replace('Children', 'Childrens')
master_df['groupname'] = master_df['groupname'].str.replace('ICAT6600', 'ICAT 6600')
master_df['groupname'] = master_df['groupname'].str.replace('Refugee', 'Refuge')
master_df['groupname'] = master_df['groupname'].str.replace('Refuge', 'Refugee')
master_df['groupname'] = master_df['groupname'].str.replace('Netcall HMS', 'HMS')
master_df['groupname'] = master_df['groupname'].str.replace('HMS', 'Netcall HMS')
master_df['groupname'] = master_df['groupname'].str.replace('Recovery Queue', 'Recovery')

col = master_df.pop('interactionid')
master_df.insert(0, 'interactionid', col)
col = master_df.pop('groupname')
master_df.insert(1, 'groupname', col)

master_df.sort_values(
    ['timequeued','agentid'],
    ascending=True,
    inplace=True
)

master_df.drop(
    columns=['Tags','Source_File','to_drop','Group'],
    inplace=True
)
drop = master_df[master_df['interactionid'].duplicated(keep='first')]
master_df.drop(
    drop.index,
    inplace=True
)
master_df.reset_index(
    drop='index',
    inplace=True
)


master_df['timequeued'] = pd.to_datetime(master_df['timequeued'], format='mixed', dayfirst=True)
master_df['timeended'] = pd.to_datetime(master_df['timeended'], format='mixed', dayfirst=True)
master_df['Year'] = master_df['timequeued'].dt.year
master_df['Month'] = master_df['timequeued'].dt.month

In [ ]:
final_df = pd.concat([master_df,customer_service])

values_to_change = [
    'Unknown','Completed','0'
]
for value in values_to_change:
    if value == '0':
        final_df['result'] = final_df['result'].str.replace('0','Failed')
    else:
        final_df['result'] = final_df['result'].str.replace(f'{value}','Other')

final_df['nullcount'] = final_df.isna().sum(axis=1)
final_df.sort_values(by=['interactionid','nullcount'], inplace=True)
drop = final_df[final_df['interactionid'].duplicated(keep='first')]
final_df.drop(drop.index, inplace=True)
final_df.drop(columns='nullcount', inplace=True)

mf.validate_data(final_df)

#########################################################################################################################################################################################
DataFrame: None


,interactionid,groupname,type,direction,timequeued,timeended,queue,skill,agent,customername,customercontact,queuetime,duration,talktime,holdtime,wrapupduration,agenthandlingtime,result,activitycode,assistedby,transferredto,subject,conversationstarttime,conversationendtime,conversationduration,agentusername,agentid,casereference,forwarded,source,conversationid,ipaddress,Year,Month
0,2640671,Repair CSC,Email,Inbound,2022-01-01 02:15:00,2022-01-04 16:01:00,Repairs customer e-mail,E-Mail Responses - Customer,Georgia St Rose,NaN,Gbenga.Osunkoya@lbhf.gov.uk,85:46:29,00:04:05,00:04:05,00:00:00,00:00:04,00:04:09,Replied,Repair Status,NaN,NaN,RE: Inspection item Requires Attention - (BLK)...,01/01/2022 02:15,04/01/2022 16:01,85:46:28,4861,4861.0,NaN,0.0,Email,2516337.0,NaN,2022,1
1,2640679,Repair CSC,Email,Inbound,2022-01-01 06:14:00,2022-01-04 16:05:00,Repairs Internal e-mail,E-mail Responses - Internal,Georgia St Rose,NaN,Baboucarr.Gaye@lbhf.gov.uk,81:51:19,00:03:35,00:03:34,00:00:01,00:00:09,00:03:43,Replied,Repair Status,NaN,NaN,RE: Norland House Electrical Failure (Job Ref....,01/01/2022 06:14,04/01/2022 16:05,81:51:20,4861,4861.0,NaN,0.0,Email,2516345.0,NaN,2022,1
2,2640681,Repair CSC,Email,Inbound,2022-01-01 09:08:00,2022-01-04 16:05:00,Repairs Internal e-mail,E-mail Responses - Internal,Emma Hicks,NaN,orbisreports@orbisprotect.com,78:56:42,00:03:06,00:03:06,00:00:00,00:00:16,00:03:22,Transferred,OOH - Related,NaN,Mehul Patel,[Orbis Protect] - Unallocated Cases,01/01/2022 09:08,04/01/2022 16:17,79:09:21,971,971.0,NaN,0.0,Email,2516347.0,NaN,2022,1
3,2640683,Repair CSC,Email,Inbound,2022-01-01 09:38:00,2022-01-04 16:19:00,Repairs customer e-mail,E-Mail Responses - Customer,Emma Hicks,NaN,noreply@lbhf.gov.uk,78:41:00,00:14:11,00:14:10,00:00:01,00:01:00,00:15:10,Replied,Repair Status,NaN,NaN,Form submission from: Housing repair notificat...,01/01/2022 09:38,04/01/2022 16:19,78:41:00,971,971.0,NaN,0.0,Email,2516349.0,NaN,2022,1
4,2640689,Repair CSC,Email,Inbound,2022-01-01 11:49:00,2022-01-04 16:11:00,Repairs customer e-mail,E-Mail Responses - Customer,Georgia St Rose,NaN,photobook@housemark.co.uk,76:21:50,00:05:42,00:05:42,00:00:00,00:00:03,00:05:45,Replied,Report New Repair,NaN,NaN,Inspection item Requires Attention - (BLK) 1-1...,01/01/2022 11:49,04/01/2022 16:11,76:21:50,4861,4861.0,NaN,0.0,Email,2516355.0,NaN,2022,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236299,7789413,Customer Services,Call,Inbound,2025-03-07 17:15:43,2025-03-07 17:18:39,Cost of Living Crisis,Cost of Living Crisis,Joseph Addison,,07513132149,00:00:11,00:02:45,00:02:45,00:00:00,00:00:30,00:03:15,Answered,Food Bank Referral,,,,07/03/2025 17:15:43,07/03/2025 17:18:39,00:02:56,NaN,NaN,,N/A,NaN,7418617,,2025,3
236300,7789417,Customer Services,Call,Inbound,2025-03-07 17:19:01,2025-03-07 17:22:58,Cost of Living Crisis,Cost of Living Crisis,Joseph Addison,,02087496008,00:00:19,00:03:39,00:01:04,00:02:35,00:00:30,00:04:09,Answered,Fuel Voucher New Request,,,,07/03/2025 17:19:01,07/03/2025 17:22:58,00:03:57,NaN,NaN,,N/A,NaN,7418621,,2025,3
236301,7789433,Customer Services,Call,Outbound,2025-03-07 17:25:36,2025-03-07 17:31:29,Cost of Living Crisis,Cost of Living Crisis,Joseph Addison,,02087496008,N/A,00:05:47,00:05:47,00:00:00,00:01:48,00:07:35,Answered,Fuel Voucher New Request,,,,07/03/2025 17:25:36,07/03/2025 17:31:29,00:05:53,NaN,NaN,,N/A,NaN,7418637,,2025,3
236302,7789447,Customer Services,Messaging,Inbound,2025-03-07 17:33:17,2025-03-07 17:35:51,Cost of Living Crisis,Cost of Living Crisis,Joseph Addison,Unknown Person,,00:00:03,00:02:30,00:02:30,00:00:00,00:00:00,00:02:30,Other,,,,Bot Handover,07/03/2025 17:33:17,07/03/2025 17:35:51,00:02:34,NaN,NaN,,N/A,NaN,7418651,31.94.72.121,2025,3


Unique values per field:


,Field Name,No. of Unique Values
0,interactionid,2450897
1,groupname,47
2,type,3
3,direction,2
4,timequeued,1963260
5,timeended,1970107
6,queue,186
7,skill,199
8,agent,765
9,customername,749



Number of duplicate rows:
0 

<class 'pandas.core.frame.DataFrame'>
Index: 2450897 entries, 0 to 236303
Data columns (total 34 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   interactionid          2450897 non-null  object        
 1   groupname              2450897 non-null  object        
 2   type                   2450897 non-null  object        
 3   direction              2450897 non-null  object        
 4   timequeued             2450897 non-null  datetime64[ns]
 5   timeended              2450897 non-null  datetime64[ns]
 6   queue                  2450897 non-null  object        
 7   skill                  2244308 non-null  object        
 8   agent                  2205633 non-null  object        
 9   customername           983668 non-null   object        
 10  customercontact        2444992 non-null  object        
 11  queuetime              2218422 non-null  object        
 12  dur

None


Summary statistics:


,timequeued,timeended,Year,Month
count,2450897,2450897,2.450897e+06,2.450897e+06
mean,2023-08-01 14:27:53.866472192,2023-08-01 22:09:44.503980288,2.023115e+03,6.133700e+00
min,2022-01-01 02:15:00,2022-01-04 07:40:00,2.022000e+03,1.000000e+00
25%,2022-10-07 16:32:11,2022-10-10 09:54:00,2.022000e+03,3.000000e+00
50%,2023-08-08 16:11:46,2023-08-09 09:06:13,2.023000e+03,6.000000e+00
75%,2024-05-15 13:15:09,2024-05-15 14:50:59,2.024000e+03,9.000000e+00
max,2025-03-07 17:53:11,2025-03-07 17:58:22,2.025000e+03,1.200000e+01
std,NaN,NaN,9.204126e-01,3.437640e+00


End of data validation
#########################################################################################################################################################################################



In [ ]:
mf.export_to_csv(
    final_df, 
    directory = r'C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Interactions Data', 
    df_name='Netcall Interactions Master Data'
)

Exproting Netcall Interactions Master Data to CSV...
@ C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry\Netcall Interactions Master Data.csv

Successfully exported Netcall Interactions Master Data to CSV


In [ ]:
# aggregated_interactions = final_df.groupby(['direction','type','Year','Month','groupname','queue']).agg(
#     number_of_interactions = ('interactionid', 'count')
# ).reset_index()

# mf.export_to_csv(aggregated_interactions)